<a href="https://colab.research.google.com/github/buzhangjiuzhou/DLfinal/blob/master/bert_yelpFull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 导入包
# transformer提供了一些训练好的模型，可以很方便的使用。
!pip install transformers
import tensorflow as tf
import pandas as pd
import os
import tqdm
import random 
import math
# 使用分类的模型，增加了一个head用于分类。
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
% matplotlib inline

tf.__version__

'2.2.0'

In [17]:
# 导入现成的模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)
model.summary()
model.config

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_75']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 109,486,085
Trainable params: 109,486,085
Non-trainable params: 0
_________________________________________________________________


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}


In [18]:
# 数据链接，可以在 https://course.fast.ai/datasets 找到。
# agnews数据集，类别，标题，描述。
ag_url = 'https://s3.amazonaws.com/fast-ai-nlp/yelp_review_full_csv.tgz'

In [19]:
# 下载数据，并指定此时数据集的目录
ag_zip_file = tf.keras.utils.get_file(origin=ag_url,fname='yelp_review_full_csv.tgz', extract=True)
base_dir = os.path.join(os.path.dirname(ag_zip_file), 'yelp_review_full_csv')
os.listdir(base_dir)

['readme.txt', 'test.csv', 'train.csv']

In [20]:
# 读取数据
# base_dir = ''
train = pd.read_csv(os.path.join(base_dir, 'train.csv'), header=None)
print(train)
print(len(train))
print(train.head())
train = train.sample(n=50000)
print(len(train))
test = pd.read_csv(os.path.join(base_dir, 'test.csv'), header=None)
print(len(test))
print(test.head())
# f = open(os.path.join(base_dir, 'classes.txt'))
# classes = f.readlines()
# classes = [s.strip() for s in classes]
# print(classes)

        0                                                  1
0       5  dr. goldberg offers everything i look for in a...
1       2  Unfortunately, the frustration of being Dr. Go...
2       4  Been going to Dr. Goldberg for over 10 years. ...
3       4  Got a letter in the mail last week that said D...
4       1  I don't know what Dr. Goldberg was like before...
...    ..                                                ...
649995  5  I had a sprinkler that was gushing... pipe bro...
649996  1  Phone calls always go to voicemail and message...
649997  1  Looks like all of the good reviews have gone t...
649998  5  I was able to once again rely on Yelp to provi...
649999  1  I have been using this company for 11 months. ...

[650000 rows x 2 columns]
650000
   0                                                  1
0  5  dr. goldberg offers everything i look for in a...
1  2  Unfortunately, the frustration of being Dr. Go...
2  4  Been going to Dr. Goldberg for over 10 years. ...
3  4  Got 

In [21]:
# transformers自带的tokenizer中的encoder会把一段文本进行编码，然后增加上CLS和SEP，其中CLS的id是101，SEP的编码是102,PAD是0。
# 所以  a   dog   is  not   a   table
# [cls]  a   dog   is  not   a   table  [sep]
# 101   1037  3899  2003 2025  1037  2795   102
# %pprint #让列表横过来，好看一些。
tokenizer.encode(text='a dog is not a table', padding='max_length',max_length=512)[:20]

[101, 1037, 3899, 2003, 2025, 1037, 2795, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [22]:
# 预处理数据
# 按照上面的例子把数据集中的文本进行处理，并且得到对应的labels。
max_length = 512
# 把太长的句子给截断，防止后面出问题。
max_length_temp = math.floor(max_length * 0.9)
for i in tqdm.notebook.trange(train.shape[0]):
  if len(train[1][train[1].index[i]]) > max_length_temp:
    train[1][train[1].index[i]] = train[1][train[1].index[i]][0 : max_length_temp]
for i in tqdm.notebook.trange(test.shape[0]):
  if len(test[1][i]) > max_length_temp:
    test[1][i] = test[1][i][0 : max_length_temp]
train_ids = [tokenizer.encode(text=sent, padding='max_length', max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(train[1])]
test_ids = [tokenizer.encode(text=sent, padding='max_length',max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(test[1])]
train_labels = train[0].values - 1
test_labels = test[0].values - 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [23]:
# 把数据转成tensorflow张量
# train_ids是tf.Tensor组成得列表，所以用concat组合一下就行
train_ids = tf.concat(train_ids, 0)
# 把train_mask初始化为1，然后把train_ids等于0（PAD的部分）对应的值赋为0
train_mask = tf.ones(train_ids.shape)
train_mask = tf.where(tf.math.greater(train_ids, 0), train_mask, 0)
# labels本身是numpy数组，转为tf.Tensor
train_labels = tf.convert_to_tensor(train_labels)

# 测试集的处理同理test
test_ids = tf.concat(test_ids, 0)
test_mask = tf.ones(test_ids.shape)
test_mask = tf.where(tf.math.greater(test_ids, 0), test_mask, 0)
test_labels = tf.convert_to_tensor(test_labels)

print(train_ids[0])
print(train_mask[0])
print(train_labels[0])

tf.Tensor(
[  101  1045  2123  1005  1056  2113  2054  2111  2024  9343  2182  2021
  1996  3210  2024  2467  4689  2146  1012  1045  1005  2310  2179  2171
  4435  4253  2005 16269  2073  2027  2761  5271  2037  4933  2012  1012
  1037  4121 10520  1010 13558  2009  1012   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0    

In [24]:
# 训练参数
epochs = 1
batch_size = 4
validation_rate = 0.1

In [25]:
# 训练
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [26]:
# 模型训练
model.fit(x=[train_ids, train_mask], 
     y=train_labels, 
     batch_size=batch_size, 
     epochs=epochs, 
     verbose=1, 
     callbacks=None,
     validation_split=validation_rate, 
     validation_data=None, 
     shuffle=True)

11250/11250 [==============================] - 3485s 310ms/step - loss: 1.0505 - accuracy: 0.5456 - val_loss: 0.9782 - val_accuracy: 0.5764


In [27]:
# 模型测试
model.evaluate(x=[test_ids, test_mask],
        y=test_labels, 
        batch_size=4, 
        verbose=1)

12500/12500 [==============================] - 1173s 94ms/step - loss: 0.9953 - accuracy: 0.5670


[0.9952839016914368, 0.5669800043106079]